In [ ]:
should_index = True
if should_index: # Takes approx. 9 mins to index
    import os
    import mmap
    import struct
    import time
    import pickle
    from tqdm import tqdm

    DATA_DIR = '/scratch/gilbreth/bnamikas/data/waymo_open_dataset_end_to_end_camera_v_1_0_0'

    indexes = []

    start = time.time()
    for i, fn in enumerate(tqdm([file for file in os.listdir(DATA_DIR) if '.tfrecord' in file and file.startswith('train')])):
        with open(os.path.join(DATA_DIR, fn), 'rb') as file:
            indexes.append((fn, file.tell()))
            while True:
                blenth = file.read(8)
                if len(blenth) == 0:
                    indexes.pop(-1)
                    break
                proto_len = struct.unpack('q', blenth)[0]
                file.read(proto_len+8)

                indexes.append((fn, file.tell()))

    with open('index.pkl', 'wb') as f:
        pickle.dump(indexes, f)

    print("Done indexing")


  6%|▌         | 15/263 [02:22<39:09,  9.47s/it]  


KeyboardInterrupt: 

In [ ]:
# import protobuf
import protos.e2e_pb2 as e2e_pb2
import pickle
import struct
import os
import numpy as np
from PIL import Image
from io import BytesIO

DATA_DIR = '/scratch/gilbreth/bnamikas/data/waymo_open_dataset_end_to_end_camera_v_1_0_0'


with open("index.pkl", 'rb') as f:
    indexes = pickle.load(f)

def load_idx(idx):
    frame = e2e_pb2.E2EDFrame()

    with open(os.path.join(DATA_DIR, indexes[idx][0]), 'rb') as f:
        blenth = f.read(8)
        proto_len = struct.unpack('q', blenth)[0]
        f.read(4)
        protobuff = f.read(proto_len)
        server = frame.ParseFromString(protobuff)

    print(len(frame.past_states.pos_z))

    past = np.stack([frame.past_states.pos_x, frame.past_states.pos_y, frame.past_states.vel_x, frame.past_states.vel_y, frame.past_states.accel_x, frame.past_states.accel_y], axis=-1)
    

    future = np.stack([frame.future_states.pos_x, frame.future_states.pos_y], axis=-1)


    return {'PAST':past, 'FUTURE':future}#frame.intent#np.array(list(zip(frame.future_states.pos_x, frame.future_states.pos_y, frame.future_states.pos_z)))



load_idx(99999)


0


{'PAST': array([[ 9.11132812e-01, -6.79931641e-02,  9.11132812e-01,
         -8.95705402e-01,  2.10346878e-02, -2.93274820e-01,
         -6.97642565e-04],
        [ 7.26562500e-01, -5.49316406e-02,  7.26562500e-01,
         -7.96713233e-01,  1.63470209e-02,  9.89921689e-02,
         -4.68766689e-03],
        [ 5.06835938e-01, -4.22363281e-02,  5.06835938e-01,
         -5.84413052e-01,  1.66303962e-02,  2.12300181e-01,
          2.83375382e-04],
        [ 3.31542969e-01, -3.21044922e-02,  3.31542969e-01,
         -3.22308868e-01,  1.12319738e-02,  2.62104183e-01,
         -5.39842248e-03],
        [ 2.18261719e-01, -2.51464844e-02,  2.18261719e-01,
         -7.06311315e-02,  3.87633406e-03,  2.51677752e-01,
         -7.35563971e-03],
        [ 1.70410156e-01, -2.19726562e-02,  1.70410156e-01,
         -2.36727092e-02,  5.51053137e-03,  4.69584242e-02,
          1.63419731e-03],
        [ 1.70410156e-01, -2.00195312e-02,  1.70410156e-01,
          2.31026905e-04,  5.53286169e-03,  2.3903

In [6]:
import torch
from torch.utils.data import Dataset
from protos import e2e_pb2
import pickle
import struct
import os
import numpy as np
from PIL import Image
from io import BytesIO 

class WaymoE2E(Dataset): 
    def __init__(self, indexFile = 'index.pkl', data_dir='./dataset'):
        self.DATA_DIR = data_dir

        with open(indexFile, 'rb') as f:
            self.indexes = pickle.load(f)

    def decode_img(self, img):
        return np.array(Image.open(BytesIO(img)))
        

    def __len__(self):
        return len(self.indexes)

    def __getitem__(self, idx):
        frame = e2e_pb2.E2EDFrame()

        with open(os.path.join(self.DATA_DIR, self.indexes[idx][0]), 'rb') as f:
            # pass
            f.seek(self.indexes[idx][1])
            blenth = f.read(8)
            proto_len = struct.unpack('q', blenth)[0]
            f.read(4)
            protobuff = f.read(proto_len)
            frame.ParseFromString(protobuff)

        return np.vstack([self.decode_img(images.image) for images in frame.frame.images]), np.array(list(zip(frame.future_states.pos_x, frame.future_states.pos_y, frame.future_states.pos_z)))


In [2]:
from torch.utils.data import DataLoader
import time
from tqdm import tqdm
DATA_DIR = '/scratch/gilbreth/bnamikas/data/waymo_open_dataset_end_to_end_camera_v_1_0_0'

dataset = WaymoE2E(data_dir = DATA_DIR)

loader = DataLoader(
    dataset, 
    batch_size=32,
    num_workers=4,
    shuffle=False
)

start = time.time()
for batch_of_frames in tqdm(loader):
    pass
print("Total Time:", time.time()-start)


  0%|          | 9/12990 [02:56<70:50:55, 19.65s/it] 


RuntimeError: DataLoader worker (pid(s) 1072405) exited unexpectedly